In [2]:
from playlist import optimized_pomodoro_playlist
import re

token_map = {'W': 0, 'S': 1, 'L': 2, 'PAD': 3}
max_seq_len = 15  # Choose longest plausible sequence

def make_example(duration_mins):
    result = optimized_pomodoro_playlist(f"{int(duration_mins)}:00", code_format=False)
    sequence = result['sequence']
    # Remove non Pomodoro-token chars ('+', '×', or spaces)
    sequence = re.sub(r'[^WSL]', '', sequence)
    seq = [token_map[ch] for ch in sequence] + [token_map['PAD']] * (max_seq_len - len(sequence))
    return float(duration_mins) / 360.0, seq  # normalize input

# Generate lots of training data
X = []
Y = []
for mins in range(30, 361, 2):
    x, y = make_example(mins)
    X.append([x])
    Y.append(y)

In [3]:
import torch
import torch.nn as nn

class PomodoroSeq2Seq(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=64, vocab_size=4, max_len=15):
        super().__init__()
        self.encoder = nn.Linear(input_dim, hidden_dim)
        self.decoder = nn.GRU(hidden_dim, hidden_dim, batch_first=True)
        self.head = nn.Linear(hidden_dim, vocab_size)
        self.max_len = max_len

    def forward(self, x):
        # x shape: (batch, input_dim)
        enc = torch.relu(self.encoder(x))  # (batch, hidden_dim)
        # Repeat encoding for each time step
        dec_input = enc.unsqueeze(1).repeat(1, self.max_len, 1)  # (batch, max_len, hidden_dim)
        output, _ = self.decoder(dec_input)
        logits = self.head(output)  # (batch, max_len, vocab_size)
        return logits

In [4]:
import torch.optim as optim

X_tensor = torch.tensor(X, dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.long)

model = PomodoroSeq2Seq()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss(ignore_index=token_map['PAD'])

for epoch in range(4000):
    model.train()
    optimizer.zero_grad()
    logits = model(X_tensor)
    loss = criterion(logits.view(-1, 4), Y_tensor.view(-1))
    loss.backward()
    optimizer.step()
    if (epoch+1) % 100 == 0:
        print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}")

Epoch 100: Loss = 0.6413
Epoch 200: Loss = 0.0314
Epoch 300: Loss = 0.0199
Epoch 400: Loss = 0.0172
Epoch 500: Loss = 0.0121
Epoch 600: Loss = 0.0085
Epoch 700: Loss = 0.0078
Epoch 800: Loss = 0.0057
Epoch 900: Loss = 0.0044
Epoch 1000: Loss = 0.0045
Epoch 1100: Loss = 0.0055
Epoch 1200: Loss = 0.0032
Epoch 1300: Loss = 0.0023
Epoch 1400: Loss = 0.0028
Epoch 1500: Loss = 0.0022
Epoch 1600: Loss = 0.0019
Epoch 1700: Loss = 0.0019
Epoch 1800: Loss = 0.0019
Epoch 1900: Loss = 0.0021
Epoch 2000: Loss = 0.0033
Epoch 2100: Loss = 0.0014
Epoch 2200: Loss = 0.0011
Epoch 2300: Loss = 0.0012
Epoch 2400: Loss = 0.0010
Epoch 2500: Loss = 0.0009
Epoch 2600: Loss = 0.0009
Epoch 2700: Loss = 0.0010
Epoch 2800: Loss = 0.0007
Epoch 2900: Loss = 0.0032
Epoch 3000: Loss = 0.0011
Epoch 3100: Loss = 0.0009
Epoch 3200: Loss = 0.0007
Epoch 3300: Loss = 0.0130
Epoch 3400: Loss = 0.0025
Epoch 3500: Loss = 0.0017
Epoch 3600: Loss = 0.0011
Epoch 3700: Loss = 0.0009
Epoch 3800: Loss = 0.0008
Epoch 3900: Loss = 0.

In [5]:
def decode_sequence(seq):
    inv_token_map = {v: k for k, v in token_map.items()}
    return ''.join(inv_token_map[i] for i in seq if i != token_map['PAD'])

def predict_sequence(model, duration):
    model.eval()
    with torch.no_grad():
        x = torch.tensor([[duration / 360.0]], dtype=torch.float32)
        logits = model(x)
        pred = torch.argmax(logits, dim=2).cpu().numpy()[0]
        return decode_sequence(pred)

print(predict_sequence(model, 115))


WSWSWLWSWSWSWLW
